# Preprocesamientode datos de dataset gob.bo (scraping)

En este notebook se hace una limpieza básica de un conjunto de datos obtenidos del portal único de trámites gob.bo obtenido mediante web scraping.



In [4]:
import pandas as pd
import numpy as np

df = pd.read_csv('../data/2025-03-25_tramites_gobbo.raw.csv', encoding='utf-8', sep="," )
df

,titulo,institucion,descripcion,contacto,web,es_presencial,es_en_linea,requisitos,procedimiento,num_ubicaciones,...,ultima_actualizacion,observaciones,costo_descripcion,costo_montos,costo_formas,costo_conceptos,costo_ctas_bancarias,calificacion,url,categoria
0,DECLARACIONES JURADAS - SIAT EN LÍNEA,Servicio de Impuestos Nacionales - SIN,Facilitar al contribuyente el cumplimiento de ...,informaciontributaria@impuestos.gob.bo,https://www.impuestos.gob.bo/,0,1,REQUISITOS\nContar con las credenciales de acc...,PROCEDIMIENTO\nIngresar al siguiente enlace: h...,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4,https://www.gob.bo/tramite/851,Impuestos
1,INSCRIPCIÓN AL RÉGIMEN DE REINTEGRO EN EFECTIV...,Servicio de Impuestos Nacionales - SIN,"El Régimen de Reintegro del IVA (Re-IVA), alca...",informaciontributaria@impuestos.gob.bo,https://www.impuestos.gob.bo/,0,1,REQUISITOS\nCedula de Identidad vigente.\nN° d...,PROCEDIMIENTO\nIngresar al siguiente enlace pa...,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3,https://www.gob.bo/tramite/2652,Impuestos
2,REGISTRO AL PADRÓN NACIONAL DE CONTRIBUYENTES,Servicio de Impuestos Nacionales - SIN,Las Personas Naturales y Jurídicas que efectúe...,informaciontributaria@impuestos.gob.bo,https://www.impuestos.gob.bo/,0,1,REQUISITOS INSCRIPCIÓN A TRAVÉS DEL SIAT EN LÍ...,PROCEDIMIENTO DE INSCRIPCIÓN\nIngresar al sigu...,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3,https://www.gob.bo/tramite/1367,Impuestos
3,INACTIVACIÓN DEL NIT,Servicio de Impuestos Nacionales - SIN,Procedimiento para inactivar el NIT,informaciontributaria@impuestos.gob.bo,https://www.impuestos.gob.bo/,0,1,REQUISITOS\nContar con las credenciales de acc...,PRODECIMIENTO\nIngresar al siguiente enlace ht...,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3,https://www.gob.bo/tramite/2650,Impuestos
4,VERIFICADOR DE FACTURAS,Servicio de Impuestos Nacionales - SIN,Herramienta habilitada para verificar si las f...,informaciontributaria@impuestos.gob.bo,https://www.impuestos.gob.bo/,0,1,REQUISITO\nContar la factura original.,PROCEDIMIENTO\nIngresar al siguiente enlace ht...,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4,https://www.gob.bo/tramite/2654,Impuestos
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1777,PERMISO ESPECIAL DE CIRCULACION POR EXCESO DE ...,Administradora Boliviana de Carreteras - ABC,Obtencion de permiso especial para transitar c...,wgutierrez@abc.gob.bo,www.abc.gob.bo,1,0,llenado de formulario\npoliza de buena ejecuci...,NaN,1,...,Última actualización de la Información: 16/11/...,NaN,NaN,NaN,NaN,NaN,NaN,0,https://www.gob.bo/tramite/291,Transporte
1778,PERMISO DE CIURCULACION POR EXCESO DE DIMENSIONES,Administradora Boliviana de Carreteras - ABC,Obtencion de permiso para circular con exceso ...,wgutierrez@abc.gob.bo,www.abc.gob.bo,1,0,Nota Dirigida al Gerente Regional (Mencionando...,Presentar todos los requisitos en ventanilla ú...,9,...,Última actualización de la Información: 16/11/...,Observaciones: Para mayor información de los r...,NaN,NaN,NaN,NaN,NaN,0,https://www.gob.bo/tramite/296,Transporte
1779,USO DE BALANZA,Zona Franca Comercial e Industrial de Cobija -...,Brindar el servicio de pesaje a todos los tran...,MAEZOFRACOBIJA@GMAIL.COM,https://www.zofracobija.gob.bo/,1,0,Presentarse físicamente con el transporte en i...,Presentarse físicamente con el transporte en i...,0,...,Última actualización de la Información: 10/09/...,Observaciones: El uso de balanza para los usua...,Concepto de pago: uso de balanza\nForma de pag...,Monto: 100 BS,Forma de pago: EFECTIVO,Concepto de pago: uso de balanza,N° Cuenta Bancaria (Banco Unión): -,0,https://www.gob.bo/tramite/2136,Transporte
1780,CERTIFICACIÓN DE NO SUSTANCIAS AGOTADORAS DE O...,Ministerio de Medio Ambiente y Agua - MINMAA,Otorgar la Certificación de No Sustancias Agot...,quinochavez@hotmail.com,www.mmaya.gob.bo,1,0,"Carta de Solicitud de Certificación de No SAO,...",El solicitante se apersona a oficinas de la Co...,0,...,Última actualización de la Información: 10/09/...,NaN,NaN,NaN,NaN,

In [5]:
## Duplicados

df =  df.loc[:,~df.columns.duplicated()].copy()
print(df.shape)
print(df.dtypes)

df = df.drop_duplicates(subset=['titulo'], keep='last')
df = df.drop_duplicates(subset=['url'], keep='last')
df.shape

(1782, 22)
titulo                  object
institucion             object
descripcion             object
contacto                object
web                     object
es_presencial            int64
es_en_linea              int64
requisitos              object
procedimiento           object
num_ubicaciones          int64
ubicaciones             object
info_adicional          object
ultima_actualizacion    object
observaciones           object
costo_descripcion       object
costo_montos            object
costo_formas            object
costo_conceptos         object
costo_ctas_bancarias    object
calificacion             int64
url                     object
categoria               object
dtype: object


(1268, 22)

## Quitando , adicionales

Durante el scraping para concatenar los campos con múltiples entradas se usó `,` como caracter separador y se dejó una coma adicional.

In [6]:
pd.set_option('display.max_colwidth', 180)

for row in df['ubicaciones'].sample(15):
    print(f'{row} ---\n')

nan ---

nan ---

 Departamento:  Chuquisaca   Dirección:  CALLE KILÓMETRO 7, No. 366 CASI ESQUINA URRIOLAGOITIA, ZONA PARQUE BOLÍVAR - CHUQUISACA
                          Teléfono:  72005873
                         Horario de atención:   Turno Mañana:
                             Hora Inicio:  08:30 
                             Hora Fin:  12:30
                           Turno Tarde:
                               Hora Inicio:  14:30 
                               Hora Fin:  18:30
                           Ver , Departamento:  La Paz   Dirección:  Av. Montes, No 515, entre Av. Uruguay y C. Batallón Illimani, La Paz, Bolivia.
                          Teléfono:  2115700,2119276,2119251
                         Horario de atención:   Turno Mañana:
                             Hora Inicio:  08:30 
                             Hora Fin:  12:30
                           Turno Tarde:
                               Hora Inicio:  14:30 
                               Hora Fin:  18:30
  

In [7]:
columns = df.columns
numeric_cols = ['es_presencial', 'es_en_linea','num_ubicaciones', 'calificacion']

for col in columns:
    if col in numeric_cols:
        continue
    df[col] = df[col].str.rstrip(',')

df.sample(10)

,titulo,institucion,descripcion,contacto,web,es_presencial,es_en_linea,requisitos,procedimiento,num_ubicaciones,...,ultima_actualizacion,observaciones,costo_descripcion,costo_montos,costo_formas,costo_conceptos,costo_ctas_bancarias,calificacion,url,categoria
1665,"RESOLUCIÓN ADMINISTRATIVA DE APERTURA Y FUNCIONAMIENTO DE AMBULANCIAS PÚBLICAS, PRIVADAS Y SEGURIDAD SOCIAL",Gobierno Autónomo Departamental de Cochabamba - GADC,"Regular el control y el funcionamieto de las ambulancias publicas, privadas y seguridad social",gobernacion@gobernaciondecochabamba.bo,www.gobernaciondecochabamba.bo,1,0,Memorial dirigido al Director del SEDES solicitando la apertura de establecimiento de salud\n\nDocumentación de los profesionales\nFotocopia del diploma académico.\nFotocopia d...,"Se presenta toda la documentación requerida en secretairia de dirección en un folder rojo, a los dos días el interesado debe presentarse ente el responsable del centro coordina...",0,...,Última actualización de la Información: 10/09/2019 12:19,NaN,NaN,NaN,NaN,NaN,NaN,0,https://www.gob.bo/tramite/2050,Salud
1197,RECTIFICACIÓN DE DATOS EN EL REGISTRO ESTATAL DE COOPERATIVAS (REC),Autoridad de Fiscalización y Control de Cooperativas - AFCOOP,"Rectificar los errores que obren en registros o documentos públicos emitidos por la AFCOOP integrantes del Registro Estatal de Cooperativas (REC), mediante la aportación de los...",wgutierrez@afcoop.gob.bo,www.afcoop.gob.bo,1,0,"dirigida al Director(a) General Ejecutivo de la Autoridad de Fiscalización y Control de Cooperativas, AFCOOP, suscrita y firmada por el Presidente del Consejo de Administración...","La solicitud será presentada en Plataforma de Atención de la AFCOOP, dirigida al Director(a) General Ejecutivo de la Autoridad de Fiscalización y Control de Cooperativas - AFCO...",7,...,Última actualización de la Información: 25/04/2023 15:33,NaN,Concepto de pago: Autoridad de Fiscalización y Control de Cooperativas\nForma de pago: EFECTIVO\nN° Cuenta Bancaria (Banco Unión): 10000017684776\nMonto: 200 BS,Monto: 200 BS,Forma de pago: EFECTIVO,Concepto de pago: Autoridad de Fiscalización y Control de Cooperativas,N° Cuenta Bancaria (Banco Unión): 10000017684776,0,https://www.gob.bo/tramite/2956,Gobierno
1025,SOLICITUD DE PENSIÓN POR MUERTE,Gestora Pública de la Seguridad Social de Largo Plazo - GESTORA,"La Pensión por Muerte es un beneficio que se otorga a los Derechohabientes de Primer o Segundo Grado en orden de prelación de forma vitalicia y/o temporal, al fallecimiento del...",contacto@gestora.bo,https://www.gestora.bo,1,0,"DOCUMENTOS ASEGURADO:\nFotocopia Legible de Documento de Identidad (Vigente o Caduco).\nCertificado de Nacimiento, en Original o Fotocopia Legible.\nCertificado de Defunción em...","El Solicitante deben dirigirse con toda la documentación a plataforma de Atención al Cliente de la Gestora Publica de la Seguridad Social de Largo Plazo – Gestora, en la ciudad...",31,...,Última actualización de la Información: 27/04/2023 16:26,Observaciones: La duración del trámite está en función a norma vigente y características de cada caso.,NaN,NaN,NaN,NaN,NaN,0,https://www.gob.bo/tramite/2947,Gobierno
528,"REPOSICIÓN, RECTIFICACIÓN Y EXCLUSIÓN DE NOTAS EN LIBROS CENTRALIZADORES Y/O BOLETINES",Dirección Departamental de Educación La Paz - DDE-LPZ,Resolución Administrativa que autoriza la corrección de datos,Sin correo electrónico,,1,0,El trámite deberá ser realizado por el Director actual de la Unidad Educativa y para ello deberá presentar los siguientes documentos: \n\n1. Informe de observaciones de la D.D....,El trámite deberá ser realizado por el Director actual de la Unidad Educativa y para ello deberá presentar los siguientes documentos: \n\n1. Informe de observaciones de la D.D....,0,...,Última actualización de la Información: 10/09/2019 12:18,NaN,"Concepto de pago: Folder Celeste\nForma de pago: DEPOSITO\nN° Cuenta Bancaria (Banco Unión): 1-6080117\nMonto: 10 BS,Concepto de pago: Resolución Administrativa\nForma

In [8]:
for row in df['ubicaciones'].sample(12):
    print(f'{row} ---\n')

nan ---

nan ---

nan ---

nan ---

    Dirección:  Av. Panamericana km 3.5, Zona Morros Blancos
                          
                         Horario de atención:  
                             Hora Inicio:  07:30 
                             Hora Fin:  15:30
                           Ver  ---

    Dirección:  Calle Reyes Ortiz Edificio Gundlach - Torre Oeste
                          Teléfono:  22188400
                         Horario de atención:  
                             Hora Inicio:  08:30 
                             Hora Fin:  16:30
                           Ver ,    Dirección:  Beni - Magdalena (C.Nataniel Garcia Chavez S/N entre Av.6 de agosto y calle 18 de noviembre)
                          Teléfono:  800-10-1610,71132917
                         Horario de atención:  
                             Hora Inicio:  07:30 
                             Hora Fin:  15:30
                           Ver ,    Dirección:  Beni - Riberalta (C. Placido Molina No 240, entr

In [9]:
print(df.iloc[19])

titulo                                                                                                                                                               CONDOMINIO PACHA (LA PAZ - HUAJCHILLA)
institucion                                                                                                                                                        Agencia Estatal de Vivienda - AEVIVIENDA
descripcion             BRINDAR 170 DEPARTAMENTOS (SUPERFICIE DE 80,77 METROS CUADRADOS) DE 3 DORMITORIOS, 2 BAÑOS, COCINA-DESPENSA, SALA-COMEDOR, LAVANDERÍA. UBICACIÓN: Zona Huajchilla, Urbanización ...
contacto                                                                                                                                                                             info@aevivienda.gob.bo
web                                                                                                                                                                           https://ww

## Última actualización

El campo esta por ejemplo `Última actualización de la Información: 05/01/2023 13:29` se quiere el formato `DD-MM-AAAA hh:mm`

In [10]:
import re
df["ultima_actualizacion"] = df["ultima_actualizacion"].str \
    .replace(r"Última actualización de la Información: ", 
                               "", regex=True)

df["ultima_actualizacion"].sample(5)


481     11/01/2023 12:42
257     24/04/2023 15:58
495     11/01/2023 15:14
692     10/09/2019 12:18
1666    10/09/2019 12:18
Name: ultima_actualizacion, dtype: object

In [11]:
import dateparser

dateparser.parse('28/04/2023 15:55')

datetime.datetime(2023, 4, 28, 15, 55)

In [12]:
import dateparser
from datetime import datetime

def parse_format_date(date_text):
    if str(date_text) == 'nan':
        return date_text
    parsed = dateparser.parse(date_text)
    return parsed.strftime("%d-%m-%Y %H:%M")

df["ultima_actualizacion"] = df["ultima_actualizacion"].apply(parse_format_date)

In [13]:
df["ultima_actualizacion"].sample(5)

918     09-10-2019 12:18
1107    09-10-2019 12:18
845     01-11-2023 21:19
1630    28-09-2021 13:23
776     01-10-2023 18:34
Name: ultima_actualizacion, dtype: object

In [14]:
df.sample(5)

,titulo,institucion,descripcion,contacto,web,es_presencial,es_en_linea,requisitos,procedimiento,num_ubicaciones,...,ultima_actualizacion,observaciones,costo_descripcion,costo_montos,costo_formas,costo_conceptos,costo_ctas_bancarias,calificacion,url,categoria
768,INSCRIPCIÓN EN EL REGISTRO DOCENTE ADMINISTRATIVO (RDA) EGRESADOS DE LAS ESCUELAS SUPERIORES DE FORMACIÓN DE MAESTROS (ESFM) A PARTIR DE LA GESTIÓN 2014,Ministerio de Educación de Bolivia - MINEDU,"El registro de personal docente de Unidades Educativas y Centros de Educación Alternativa y Especial de egresados en gestiones anteriores al 2014, se realiza en las Direcciones...",informacion@minedu.gob.bo,www.minedu.gob.bo,1,1,"INSCRIPCIÓN EN EL REGISTRO DOCENTE ADMINISTRATIVO\n1. Original del Certificado de Nacimiento, emitido por el Servicio de Registro Cívico (SERECI)\n2. Fotocopia de Cédula de Ide...","El interesada (o) Presenta los requisitos del trámite de inscripción al RDA, en sobre manila tamaño oficio, a la Dirección Departamental de Educación correspondiente o ESFM, se...",8,...,08-01-2022 11:18,NaN,NaN,NaN,NaN,NaN,NaN,0,https://www.gob.bo/tramite/548,Educación
1125,SOLICITUD DE PLANIFICACIÓN DE DIÉSEL,Administración de Servicios Portuarios – Bolivia - ASP-B,Solicitud de planificación para despacho de Diésel.,sistemas@aspb.gob.bo,https://www.aspb.gob.bo,1,0,R-0543 Solicitud de Planificación de Diesel,Enviar el formulario el r-0543 por vía email y presentar el mismo. Debidamente llenado y firmado por el representante de la empresa de transporte.,1,...,01-12-2021 15:23,NaN,NaN,NaN,NaN,NaN,NaN,0,https://www.gob.bo/tramite/2640,Gobierno
1681,AFILIACIÓN DE ESPOSA,Seguro Social Universitario de Tarija - SSUTAR,Realizar la afiliación de Esposa o Esposo,ssutarija@ssutarija.org.bo,www.ssutarija.org.bo,1,0,REQUISITOS PARA BENEFICIARIA ESPOSA (O)\n\n1.-Solicitud de Afiliaciónde beneficiario dirigida a Gerencia General\n\n2.-Certificado denacimiento original computarizado\n\n3.-Cer...,Presentada la Documentación del 1 al 17 de concluye el tramite en 6 días Habiles,1,...,28-04-2023 14:26,"Observaciones: Los requisitos del 15 al 17 son variables de acuerdo disponibilidad de Trabajo Social, Asesoría legal y comité de prestaciones que se reúne una vez al mes",Concepto de pago: pago de valores\nForma de pago: EFECTIVO\nN° Cuenta Bancaria (Banco Unión): -\nMonto: 18 BS,Monto: 18 BS,Forma de pago: EFECTIVO,Concepto de pago: pago de valores,N° Cuenta Bancaria (Banco Unión): -,0,https://www.gob.bo/tramite/1699,Salud
1579,EMISION DE CERTIFICADO DE NACIDO VIVO,Gobierno Autónomo Departamental de Cochabamba - GADC,EMISION DE UN DUPLICADO DEL CERTIFICADO DE NACIDO VIVO,gobernacion@gobernaciondecochabamba.bo,www.gobernaciondecochabamba.bo,1,0,"1.- Pulicacion en un periodico local comunicando el extravio del certificado de nacido vivo del ñiño(a),tamaño de la publicacion 5cm x 4cm., conel contenido siguiente (""SE ...",Entregar los 4 requisitos en secretaria dedireccion de esta institucion.\n\nRegresaren 2 dias habiles para recoger el certificado con su carnet de identidad.,0,...,09-10-2019 12:18,Observaciones: La emision del certificado de nacido vivo es gratiuta.,NaN,NaN,NaN,NaN,NaN,4,https://www.gob.bo/tramite/1979,Salud
1460,CESACIÓN O CANCELACIÓN DE LA PERMANENCIA INDEFINIDA,Dirección General de Migración - DIGEMIG,"Acto Administrativo emitido por la Dirección General de Migración, como efecto de la cesación o cancelación de la condición de refugio emitido por la CONARE",comunicacion@migracion.gob.bo,http://www.migracion.gob.bo/,1,0,"1. Resolución Administrativa de cesación, cancelación o revocatoriade condición de refugio.\n\n2. Cédula de Identidad (original), expedida por el SEGIP de la persona que regist...","En la DIGEMIG o en Administración Departamental\n\n1. Se hace presente en la ventanilla de inicio\n\n2. Si cumple los requisitos, se le autoriza el pago en Oficinas del Banco U...",0,...,09-10-2019 12:18,Observaciones: El trámite tiene una duración de hasta 6 días en la Oficin

## Extrayendo Duración y marco Legal de info_adicional

La variable `info_adicional` tiene registros como:

```
Duración: 5 dias
Marco Legal:
Ley 025
Última actualización de la Información: 10/09/2019 12:18
```



In [15]:
duracion_re = re.compile(r"(.)*Duración: (?P<duracion>\d+\ [\w]+)")
marcolegal_re = re.compile(r"Marco Legal:\n(?P<ley>.+)")

def extraer_duracion_y_marcolegal(info_adicional):
    if str(info_adicional) == 'nan':
        return info_adicional
    duraciones = []
    marco_legal = []
    lineas = info_adicional.split('\n')
    for i, linea in enumerate(lineas):
        matches = list(re.finditer(duracion_re, linea))
        if len(matches) > 0:
            obtained = matches[0].groupdict()
            duraciones.append(obtained['duracion'])
    matches = list(re.finditer(marcolegal_re, info_adicional))
    if len(matches) > 0:
        for match in matches:
            marco_legal.append(match['ley'])
    return duraciones, marco_legal

test_str = '''
Duración: 5 dias
Marco Legal:
Ley 025
Última actualización de la Información: 10/09/2019 12:18,Duración: 5 dias
Marco Legal:
Ley 025
Última actualización de la Información: 10/09/2019 12:18
'''

extraer_duracion_y_marcolegal(test_str)

(['5 dias', '5 dias'], ['Ley 025', 'Ley 025'])

In [16]:
duracion_re = re.compile(r"(.)*Duración: (?P<duracion>\d+\ [\w]+)")

def extraer_duracion(info_adicional):
    if str(info_adicional) == 'nan':
        return info_adicional
    duraciones = []
    lineas = info_adicional.split('\n')
    for i, linea in enumerate(lineas):
        matches = list(re.finditer(duracion_re, linea))
        if len(matches) > 0:
            duraciones.append(matches[0].groupdict()['duracion'])
    return ','.join(duraciones)

In [17]:
df['duracion'] = df['info_adicional'].apply(extraer_duracion)
df['duracion']

0                 
1                 
2                 
3                 
4                 
           ...    
1777              
1778        3 dias
1779    20 minutos
1780       10 dias
1781       10 dias
Name: duracion, Length: 1268, dtype: object

In [18]:
df.sample(3)[['titulo','info_adicional', 'duracion']]

,titulo,info_adicional,duracion
1072,OBTENCIÓN DE CREDENCIALES PARA EL EMPLEADOR,Marco Legal:\nLey N°065 de pensiones,
1593,TRAMITE DE AFILIACIÓN,Duración: 1 día\nMarco Legal:\nCódigo de Seguridad Social\nReglamentos emitidos por la ASUS\nÚltima actualización de la Información: 14/04/2023 18:44,1 día
1537,BAILES POR FECHAS ESPECIALES (UTILIZAR ESPACIOS PÚBLICOS),Duración: 3 dias\nMarco Legal:\nOrdenanza Municipal Nº46/2001\nOrdenanza Municipal Nº 23/2006\nObservaciones: ¿DÓNDE PUEDO REALIZAR EL TRÁMITE? En la Dirección de Medio Ambient...,3 dias


In [19]:
marcolegal_re = re.compile(r"Marco Legal:\n(?P<ley>.+)")

def extraer_marco_legal(info_adicional):
    if str(info_adicional) == 'nan':
        return info_adicional
    marco_legal = []
    matches = list(re.finditer(marcolegal_re, info_adicional))
    if len(matches) > 0:
        for match in matches:
            marco_legal.append(match['ley'])
    return ','.join(marco_legal)

df['marco_legal'] = df['info_adicional'].apply(extraer_marco_legal)


In [20]:
df.sample(3)[['titulo','info_adicional', 'duracion', 'marco_legal']]

,titulo,info_adicional,duracion,marco_legal
1735,INSPECCIÓN DE KITS DE CONVERSIÓN A GNV,"Duración: 3 dias\nMarco Legal:\nDecreto Supremo N° 572, de 14 de julio de 2010\nISO 2859-1\nISO 15500\nÚltima actualización de la Información: 10/09/2019 12:18",3 dias,"Decreto Supremo N° 572, de 14 de julio de 2010"
1131,REGISTRO PROVISIONAL,Marco Legal:\nD.S. 26256\nÚltima actualización de la Información: 26/04/2023 15:51,,D.S. 26256
1546,FORMULARIO EXPLORACIÓN MINERA DE ACTIVIDADES PEQUEÑAS - EMAP (SECTOR MINERÍA),"Duración: 15 dias\nMarco Legal:\nEl Formulario de Exploracion Minera de Actividades pequeñas, ser rige de acuerdo con los niveles señalados en el Artículo 25 de la Ley N° 1333 ...",15 dias,"El Formulario de Exploracion Minera de Actividades pequeñas, ser rige de acuerdo con los niveles señalados en el Artículo 25 de la Ley N° 1333 de Medio Ambiente, de 27 de abril..."


## Cuenta bancaria

Algunos valores de la columna `costo_ctas_bancarias` son como: `N° Cuenta Bancaria (Banco Unión):,N° Cuenta Bancaria (Banco Unión):,` (no tienen cuenta) y otros `N° Cuenta Bancaria (Banco Unión): 1-4668220 (Banco Union),N° Cuenta Bancaria (Banco Unión): 1-293633 (Banco Union),N° Cuenta Bancaria (Banco Unión): 1-4668220 (Banco Union),N° Cuenta Bancaria (Banco Unión): 1-4668220 (Banco Union),`. Se debería quitar los valores `,N° Cuenta Bancaria (<Banco>)`

In [21]:
df['costo_ctas_bancarias']

0                                       NaN
1                                       NaN
2                                       NaN
3                                       NaN
4                                       NaN
                       ...                 
1777                                    NaN
1778                                    NaN
1779    N° Cuenta Bancaria (Banco Unión): -
1780                                    NaN
1781                                    NaN
Name: costo_ctas_bancarias, Length: 1268, dtype: object

In [22]:
ctas_re = re.compile(r'N° Cuenta Bancaria \((?P<banco>[A-Za-z_ÑñÁáÉéÍíÓóÚú ]+)\):\ ?,?(?P<cta>[\d\-]+)?')

def extraer_ctas_bancarias_y_bancos(texto):
    if str(texto) == 'nan':
        return {}
    ctas = []
    bancos = []
    matches = list(re.finditer(ctas_re, texto))
    if len(matches) > 0:
        for match in matches:
            _d = match.groupdict()
            # print(_d)
            ctas.append(_d['cta'] if _d.get('cta', None) is not None else '')
            bancos.append(_d['banco'] if _d.get('banco', None) is not None else '')
    return {
        'cuentas': ','.join(ctas),
        'bancos': ','.join(bancos)
    }


tests = [
    'N° Cuenta Bancaria (Banco Unión): 1-4668220 (Banco Union),N° Cuenta Bancaria (Banco Unión): 1-293633 (Banco Union),N° Cuenta Bancaria (Banco Unión): 1-4668220 (Banco Union),N° Cuenta Bancaria (Banco Unión): 1-4668220 (Banco Union),',
    'N° Cuenta Bancaria (Banco Unión):,N° Cuenta Bancaria (Banco Unión):,' '', 'algo', 'nan']

for test in tests:
    print(extraer_ctas_bancarias_y_bancos(test))


{'cuentas': '1-4668220,1-293633,1-4668220,1-4668220', 'bancos': 'Banco Unión,Banco Unión,Banco Unión,Banco Unión'}
{'cuentas': ',', 'bancos': 'Banco Unión,Banco Unión'}
{'cuentas': '', 'bancos': ''}
{}


In [23]:
s1 = df['costo_ctas_bancarias'].apply(lambda x: extraer_ctas_bancarias_y_bancos(x).get('cuentas', ''))
s1.unique()

array(['', '-',
       '10000004670150,10000004670150,10000004670150,10000004670150,10000004670150,10000004670150,10000004670150,10000004670150,10000004670150,10000004670150',
       '10000012470724', '10000012470724,10000012470724',
       '10000030880666', '1-6036425,1-6036425',
       '1-6036425,1-6036425,1-6036425,1-6036425,1-6036425,1-6036425,1-6036425,1-6036425,1-6036425,1-6036425,1-6036425,1-6036425,1-6036425,1-6036425',
       '1-4668220,1-293633', '1-4668220,1-4668220,1-4668220,1-293633',
       '10000004671306', '10000029669477', '1-4668220,1-4668220',
       '1-4668220,1-293633,1-4668220,1-4668220', '1-3400897',
       '1-293633,1-4668220', '1-33175642', '1-4668212', '1-',
       '1-33175642,', '1-33175642,,',
       '1-295077,1-295077,1-295077,1-295077,1-295077,1-295077,1-295077',
       '1-295077', '1-345688', ',', '1000004669343', '10000004669343,',
       '119980908,', '14041070', '10000004669343', '10000004669343,-',
       '1-6080117,1-6080117', '10000004713083',
     

In [24]:
s2 = df['costo_ctas_bancarias'].apply(lambda x: extraer_ctas_bancarias_y_bancos(x).get('bancos', ''))
s2.unique()

array(['', 'Banco Unión',
       'Banco Unión,Banco Unión,Banco Unión,Banco Unión,Banco Unión,Banco Unión,Banco Unión,Banco Unión,Banco Unión,Banco Unión',
       'Banco Unión,Banco Unión',
       'Banco Unión,Banco Unión,Banco Unión,Banco Unión,Banco Unión,Banco Unión,Banco Unión,Banco Unión,Banco Unión,Banco Unión,Banco Unión,Banco Unión,Banco Unión,Banco Unión',
       'Banco Unión,Banco Unión,Banco Unión,Banco Unión',
       'Banco Unión,Banco Unión,Banco Unión',
       'Banco Unión,Banco Unión,Banco Unión,Banco Unión,Banco Unión,Banco Unión,Banco Unión',
       'Banco Unión,Banco Unión,Banco Unión,Banco Unión,Banco Unión',
       'Banco Unión,Banco Unión,Banco Unión,Banco Unión,Banco Unión,Banco Unión',
       'Banco Unión,Banco Unión,Banco Unión,Banco Unión,Banco Unión,Banco Unión,Banco Unión,Banco Unión,Banco Unión',
       'Banco Unión,Banco Unión,Banco Unión,Banco Unión,Banco Unión,Banco Unión,Banco Unión,Banco Unión',
       'Banco Unión,Banco Unión,Banco Unión,Banco Unión,Ba

In [25]:
df['costo_ctas_bancarias'] = s1
df['costo_bancos'] = s2
df.sample(10)

,titulo,institucion,descripcion,contacto,web,es_presencial,es_en_linea,requisitos,procedimiento,num_ubicaciones,...,costo_montos,costo_formas,costo_conceptos,costo_ctas_bancarias,calificacion,url,categoria,duracion,marco_legal,costo_bancos
1467,OTORGACIÓN Y REGISTRO DE PERSONALIDAD JURÍDICA A ENTES DE COORDINACIÓN CON ENTIDADES CONFORMADAS POR PERSONAS COLECTIVAS:,Ministerio de la Presidencia - MINPRES,"Registrar y otorgar a entes de coordinación conformadas por personas colectivas, a solicitud su personalidad jurídica.",Sin correo electrónico,http://www.mpr.gob.es/,1,0,PARA:\n\na) ORGANIZACIONES SOCIALES Y ENTIDADES CIVILES SIN FINES DE LUCRO\n\nEn original:\n\n\n1.-solicitud de otorgación de personalidad jurídica formulario B - (...,1. Entes de Coordinación conformada por personas Colectivas o persona colectiva Adjunta todos los requisitos para ingresar su trámite.\n\n2.-En ventanilla de la unidad de perso...,0,...,"Monto: 3600 BS,Monto: 15000 BS,Monto: 15000 BS","Forma de pago: DEPOSITO,Forma de pago: DEPOSITO,Forma de pago: DEPOSITO","Concepto de pago: cancelacion por registro de personalidad juridica para entes de coordinacion,Concepto de pago: cancelacion por registro de personalidad juridica para entes de...","1-24364270,1-24364270,1-24364270",0,https://www.gob.bo/tramite/1404,Justicia,2 meses,Observaciones: Nota.-En todos los casos de solicitud de personalidad Jurídica el procedimiento y plazos son los mismos. Solo hay una variación en los costos y requisitos(sea pe...,"Banco Unión,Banco Unión,Banco Unión"
1496,ACCIÓN DE INFRACCIÓN A DERECHOS DE PROPIEDAD INDUSTRIAL PARA PERSONAS NATURALES BOLIVIANAS.,Servicio Nacional de Propiedad Intelectual - SENAPI,"Una acción de infracción es una acción legal ejercida por el titular de un registro de propiedad industrial contra quienes, sin autorización de los titulares, realizan actos qu...",plataforma@senapi.gob.bo,www.senapi.gob.bo,1,0,Depósitos bancarios\nEscrito de acción de infracción \nPrueba que considere pertinente\n,Pago correspondiente según arancel a realizarse en el Banco Unión.\n\nPresentación en Ventanilla Única del SENAPI en un folder con todos los requisitos.\n\nEn caso de observaci...,1,...,"Monto: 250 BS,Monto: 50 BS","Forma de pago: DEPOSITO,Forma de pago: DEPOSITO","Concepto de pago: Infracción,Concepto de pago: Notificacion","1-4668220,1-4668220",0,https://www.gob.bo/tramite/2515,Justicia,,Decisión de la CAN N° 486 (Régimen Común sobre Propiedad Industrial). - .,"Banco Unión,Banco Unión"
8,RECAUDACION DEL IMPUESTO DEPARTAMENTAL SOBRE LAS TRANSMISIONES SUCESORIAS Y GRATUITAS DE BIENES (IDTSG),Gobierno Autónomo Departamental de Cochabamba - GADC,REALIZAR LA RECAUDACION POR COBRO DEL IMPUESTO DEPARTAMENTAL SOBRE LAS TRANSMISIONES SUCESORIAS Y GRATUITAS DE BIENES (IDTSG).,gobernacion@gobernaciondecochabamba.bo,www.gobernaciondecochabamba.bo,1,0,LOSREQUISITOS SON LO SIGUIENTES; (FOTOCOPIA Y DOCUMENTO ORIGINAL)\n\nInmuebles:\n\n-Folio Real (actualizado)\n\n-Documento de Transmision.\n\n-Comprobante del pago del impuesto...,- Conla documentacion citada (en el cuadro de requisitos) en original y fotocopia elcontribuyente debe apersonarse al area de atencion al contribuyente a fin derealizar la liqu...,0,...,NaN,NaN,NaN,,0,https://www.gob.bo/tramite/2081,Impuestos,2 horas,"LA RECAUDACION DEL IMPUESTO DEPARTAMENTAL SOBRE LAS TRANSMISIONES SUCESORIAS Y GRATUITAS DE BIENES (IDTSG), SE LO REALIZA EN AMPARO A LA LEY N° 450 Y DECRETO DEPARTAMENTAL REGL...",
235,REGISTRO OCE,Aduana Nacional - AN,Registrar operadores de comercio exterior,ccastro@aduana.gob.bo,https://www.aduana.gob.bo,1,1,cédula de identidad,NaN,1,...,NaN,NaN,NaN,,0,https://www.gob.bo/tramite/260,Económico-Productivo,,Ley 1990,
661,COPIA DE DIPLOMA DE BACHILLER EN HUMANIDADES,Universidad Amazónica de Pando - UAP,NaN,carloshuanca@gmail.com,,1,0,NaN,NaN,0,...,NaN,NaN,NaN,,0,https://www.gob.bo/tramite/2570,Educación,,Última actualización de la Información: 10/09/2019 12:18,
259,RECLAMOS RELACIONADOS A OPERACIONE

## Montos

La columna `costo_montos` tiene valores de la forma `Monto: <cantidad> <moneda>` Con lo siguiente se extraerá `<cantidad> <moneda>`. Ejemplos

- `Monto: 400 BS,Monto: 200 BS,Monto: 500 BS,Monto: 500 BS` --> `400 BS, 500 BS, 500 BS`.

In [26]:
monto_re = re.compile(r'Monto: (?P<monto>[\d.]+ \w+),?')

def extraer_monto(texto):
    if str(texto) == 'nan':
        return ''
    montos = []
    matches = list(re.finditer(monto_re, texto))
    if len(matches) == 0:
        return ''
    for match in matches:
        _d = match.groupdict()
        # print(_d)
        montos.append(_d['monto'] if _d.get('monto', None) is not None else '')
    return ','.join(montos)

tests = [
    'Monto: 0 UFV,Monto: 100 UFV',
    'Monto: 36 BS',
    'Monto: 6.112 BS,Monto: 3.056 BS,Monto: 3.358 BS,Monto: 6.716 BS,Monto: 2.216 BS,Monto: 3.361 BS,Monto: 6.723 BS,Monto: 3.693 BS,Monto: 7.388 BS,Monto: 2.014 BS',
]

for test in tests:
    print(test)
    print(extraer_monto(test))

Monto: 0 UFV,Monto: 100 UFV
0 UFV,100 UFV
Monto: 36 BS
36 BS
Monto: 6.112 BS,Monto: 3.056 BS,Monto: 3.358 BS,Monto: 6.716 BS,Monto: 2.216 BS,Monto: 3.361 BS,Monto: 6.723 BS,Monto: 3.693 BS,Monto: 7.388 BS,Monto: 2.014 BS
6.112 BS,3.056 BS,3.358 BS,6.716 BS,2.216 BS,3.361 BS,6.723 BS,3.693 BS,7.388 BS,2.014 BS


In [27]:
df['costo_montos'] = df['costo_montos'].apply(extraer_monto)
df.sample(10)[['titulo','costo_montos']]

,titulo,costo_montos
1561,SOLICITUD DE CERTIFICADO DE DISPENSACION - SCD (SECTOR ELECTRICIDAD CATEGORIA 4).,
433,FOTOC. TIT. PROV. NAL. - APOSTILLA,199 BS
782,LEGALIZACIÓN RESOLUCIÓN MINISTERIAL O ADMINISTRATIVA DE AUXILIAR DE ENFERMERÍA,38 BS
471,TRÁMITE ESPECIAL A,61 BS
664,ADICIÓN O SUPRESIÓN DE NOMBRE (S) Y/O APELLIDO (S) EN DIPLOMA DE BACHILLER,10 BS
908,CERTIFICADO DE ACCESO A PROGRAMA DE POSGRADO,
1734,HABILITACIÓN DE TALLERES EN RECINTOS ADUANEROS Y ZONAS FRANCAS DE EMISIÓN DE GASES,"400 BS,300 BS"
763,DIPLOMA DE BACHILLER DE PERSONAS QUE ESTUDIARON EN EL EXTERIOR,
1674,AFILIACIÓN DE ASEGURADO TITULAR O TRABAJADOR.,15 BS
90,SOLICITUD DE AUTORIZACIÓN DE SALIDA DE OBRAS DE ARTE CONTEMPORÁNEAS,"40 BS,8 BS"


## Ubicaciones

En la columna `ubicaciones` el texto contiene `Direcciones: ` inecesariamente. Primero se quitará esto. También la palabra "Ver"

In [28]:
df['ubicaciones'] = df['ubicaciones'].str.replace('Direcciones: ', '')
df['ubicaciones'] = df['ubicaciones'].str.replace('Ver ', '')
df.sample(10)[['titulo','ubicaciones']]

,titulo,ubicaciones
758,SECUENCIA DE ESTUDIOS,Dirección: Calle Ecuador Nº E-442 Telf: 4228777\n Teléfono: 4228777\n Horario de atención: \n ...
658,CERTIFICADO DE CAPACITACIÓN,"Dirección: AVENIDA VICTOR PAZ NUMERO 260 ENTRE LA CALLE MENDEZ Y SUIPACHA\n Teléfono: 6635633,6643194,6643487,6641488\n ..."
1199,TRAMITE NRO. 33 ~ EMANCIPACIONES Y/O HABILITACIONES,NaN
922,OTROS DOCUMENTOS ACADÉMICOS - ADMINISTRATIVOS,NaN
191,REGISTRO DE NÚMERO DE IDENTIFICACIÓN DE AGENTE DE RETENCIÓN (N.I.A.R.),"Departamento: La Paz Dirección: OFICINA NACIONAL: Av. Mcal. Santa Cruz Esq. Calle Yanacocha Edif. Hansa Piso 11\n Teléfono: 2-2112186,2-2154523..."
589,OTROS DOCUMENTOS ACADÉMICOS,"Departamento: La Paz Dirección: Ministerio de Educación La Paz, Av. Arce No. 2147\n \n Horario de atención: Turno Mañana..."
947,ACTA DE DEFENSA DE MODALIDAD DE GRADUACIÓN,NaN
460,"REGISTRO DE OBRAS ARTÍSTICAS (CINEMATOGRÁFICAS), PARA PERSONAS NATURALES Y JURIDICAS BOLIVIANAS.","Departamento: La Paz Dirección: Av. Montes, No 515, entre Av. Uruguay y C. Batallón Illimani, La Paz, Bolivia.\n Teléfono: 2115700,2119276,211925..."
1736,DOCUMENTO DE IDONEIDAD - REVALIDACIÓN,NaN
1534,INSCRIPCIÓN EN EL REGISTRO NACIONAL DE CONSULTORÍA AMBIENTAL RENCA (UNIDADES AMBIENTALES DE INSTANCIAS PÚBLICAS O PRIVADAS),NaN


In [29]:
df["ultima_actualizacion"].str \
    .replace(r"Última actualización de la Información: ", 
                               "", regex=True)

0                    NaN
1                    NaN
2                    NaN
3                    NaN
4                    NaN
              ...       
1777    16-11-2021 12:15
1778    16-11-2021 12:14
1779    09-10-2019 12:19
1780    09-10-2019 12:18
1781    09-10-2019 12:18
Name: ultima_actualizacion, Length: 1268, dtype: object

## Horraio de atención

De la misma columna `ubicaciones` se podría extraer los horarios de atención. Por ejemplo

```
    Dirección:  La plata y Bolivar
                          Teléfono:  +59125252791
                         Horario de atención:  
                             Hora Inicio:  08:00 
                             Hora Fin:  16:00
                           Ver
```

Se puede extraer. 8:00 - 16:00.



In [30]:
hora_inicio_re = re.compile(r' *Hora Inicio: +(?P<inicio>[\d]+:[\d]+) *')
hora_fin_re = re.compile(r' *Hora Fin: +(?P<fin>[\d]+:[\d]+)')

def extraer_regex_con_grupo(regex,texto, prop):
    if str(texto) == 'nan':
        return ''
    results = []
    matches = list(re.finditer(regex, texto))
    if len(matches) == 0:
        return ''
    for match in matches:
        _d = match.groupdict()
        results.append(_d[prop] if _d.get(prop, None) is not None else '')
    return ','.join(results)


def extraer_horarios(text):
    inicios = extraer_regex_con_grupo(hora_inicio_re, text, 'inicio').split(',')
    fines = extraer_regex_con_grupo(hora_fin_re, text, 'fin').split(',')
    horarios = []
    if inicios[0] == '':
        return ''
    for i, inicio in enumerate(inicios):
        horarios.append(f'{inicio} - {fines[i]}')
    return ','.join(horarios)


tests = ['''    Dirección:  En la ciudad de La Paz, Jerarquico Av. Víctor Sanjinés Nº 2705 esq. Calle Méndez Arcos Plaza España
                          Teléfono:  (591-2) 2412789,2412048
                         Horario de atención:   Turno Mañana:
                             Hora Inicio:  08:30 
                             Hora Fin:  12:30
                           Turno Tarde:
                               Hora Inicio:  14:30 
                               Hora Fin:  18:30
                           Ver ,    Dirección:  En ciudad de Tarija Calle Mariscal Sucre y Virginio Lema Nº 397, Edif. ECOBOL (Mezzanine)
                          Teléfono:  (591-4) 6112535
                         Horario de atención:   Turno Mañana:
                             Hora Inicio:  08:30 
                             Hora Fin:  12:30
                           Turno Tarde:
                               Hora Inicio:  14:30 
                               Hora Fin:  18:30
                           Ver ,    Dirección:  En la ciudad de La Paz, Alzada y Jerarquico Calle Arturo Borda Nº 1933, Zona Cristo Rey (Alto Sopoca
                          Teléfono:  (591-2) 2412613,2411973
                         Horario de atención:   Turno Mañana:
                             Hora Inicio:  08:30 
                             Hora Fin:  12:30
                           Turno Tarde:
                               Hora Inicio:  14:30 
                               Hora Fin:  18:30
                           Ver1
''',
'''    Dirección:  Avenida José Aguirre Acha Edificio Servicio de Impuestos Nacionales Zona Los Pinos
                          Teléfono:  2 2606060
                         Horario de atención:  
                             Hora Inicio:  07:30 
                             Hora Fin:  16:30
                           Ver ''',
         '']

for test in tests:
    print(extraer_horarios(test))



08:30 - 12:30,14:30 - 18:30,08:30 - 12:30,14:30 - 18:30,08:30 - 12:30,14:30 - 18:30
07:30 - 16:30



In [31]:
df['horarios_atencion'] = df['ubicaciones'].apply(extraer_horarios)
df.sample(10)[['titulo', 'horarios_atencion', 'ubicaciones']]

,titulo,horarios_atencion,ubicaciones
819,CERTIFICACIÓN DE CERTIFICADO DE INTERNADO ROTATORIO CON CARGA HORARIA,"08:30 - 12:30,14:30 - 18:30,08:30 - 12:30,14:30 - 18:30,08:30 - 12:30,14:30 - 18:30,08:30 - 12:30,14:30 - 18:30,08:30 - 12:30,14:30 - 18:30,08:30 - 12:30,14:30 - 18:30","Departamento: La Paz Dirección: Ministerio de Educación La Paz, Av. Arce No. 2147\n \n Horario de atención: Turno Mañana..."
1660,CERTIFICACIÓN AL MANEJO DE RESIDUOS GENERADOS EN ESTABLECIMIENTOS DE SALUD Y BIOSEGURIDAD,,NaN
1282,AUTORIZACIÓN PARA PASACALLE,08:00 - 14:00,Dirección: Segunda Entrada Barrio Nueva Trinidad (detrás de la Estación de Bomberos de la Policía) Calle N° 2\n \n Horario...
1408,REGISTRO O LEVANTAMIENTO DE ARRAIGO TEMPORAL O DEFINITIVO.,,NaN
1720,REGISTRO DE LINEAS NAVIERAS AFINES A LA ACTIVIDAD NAVIERO MERCANTE,"08:00 - 15:30,08:00 - 16:00","Dirección: Edificio Ministerio de Defensa Nº 2502\n Teléfono: 2432525,2610400\n Horario de atención: \n ..."
1613,LEGALIZACIÓN DE CARNET DE AFILIACIÓN AL IBC,,NaN
67,SOLICITUD PARA EL ARRENDAMIENTO DEL SALON AUDITORIUM DEL EDIFICIO CCLP,08:00 - 16:00,Departamento: La Paz Dirección: AV MARISCAL SANTA CRUZ NRO 1260 EDIFICIO CENTRO DE COMUNICACIONES LA PAZ SALON AUDITORIUM\n Teléfono: 2367085\n ...
536,TRÁMITE DE INSCRIPCIÓN ESCALAFÓN,"08:00 - 12:00,14:30 - 18:30","Dirección: Avenida del Maestro N° 345\n Teléfono: 6454760,6461335\n Horario de atención: Turno Mañana:\n ..."
1179,REGISTRO DE ESCISIÓN DE COOPERATIVAS,"08:30 - 12:30,14:30 - 18:30,08:30 - 12:30,14:30 - 18:30,08:30 - 12:30,14:30 - 18:30,08:30 - 12:30,14:30 - 18:30,08:30 - 12:30,14:30 - 18:30,08:30 - 12:30,14:30 - 18:30,08:30 - ...","Dirección: Calle José Saravia N°1600, esquina Calle Pioneros de Roschdale (a unos pasos del Ex Cine Ebro)\n Teléfono: 2129205,2123560\n ..."
570,CERTIFICADO DE BUENA CONDUCTA,,NaN


In [32]:
# guardando como csv
df.to_csv('../data/gobbo_tramites_pre.csv', index=False)